In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from Map import Map
from Tabu import Tabu
from TwoStage import TwoStage
from L_shaped import MasterProblem as mp
import matplotlib.pyplot as plt
import matplotlib.pylab as pl 
import matplotlib.cm as cm
import time
import networkx as nx
import matplotlib as mpl
import logging

**Modelling Parameters**

In [3]:
rides = 6
bus = 1
scenarios = 25
MIPGap = 0.001
probability = [0.7, 0.1, 0.15, 0.05]

**Defining the Map**

In [4]:
mappy = Map(rides)

In [5]:
def Mod3():
    t3 = time.time()
    lshaped = mp(
        mappy,
        bus=bus,
        scenarios=scenarios,
#         probability=probability
    )
    lshaped.initialize()
    lshaped.model.params.MIPGap = MIPGap
    lshaped.model.params.TimeLimit = 0.01
    lshaped.optimize()
    t3 = time.time() - t3
    return lshaped, t3

**Plotting Functions**

In [6]:
def plot_trend(ub,lb,base):
    x = list(range(len(ub)))
    figure = plt.figure()
    plt.plot(x,ub,label='upper-bound')
    plt.plot(x,[base]*len(ub),label='two-stage')
    plt.plot(x,lb,label='lower-bound')
    plt.annotate(int(ub[-1]),[x[-1],ub[-1]])
    plt.legend()
    plt.savefig('./figures/trend.png')
    plt.show()

colors = ['green','blue','yellow','red','pink'] 
def displaygraph(n, e, modname):
    for k in e.keys():
        for i, j in e[k]:
            plt.plot((n[i][0],n[j][0]),(n[i][1],n[j][1]), color=colors[k] ,marker='o', linewidth=2, linestyle='dashed')
    for i in n:
        plt.annotate(i,n[i],textcoords="offset points",xytext=(0,10),ha='center')
    plt.savefig('./figures/'+modname+'.png')
    plt.show()


In [7]:
lshaped, t = Mod3()

Using license file C:\Users\lavkeshg\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter MIPGap to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 0.01
   Prev: inf  Min: 0.0  Max: inf  Default: inf
-84.06669017577042 -10.1080528141878
73.95884532351084 0.00020796192822558623
-75.73824163583231 -8.103940877050348
67.63450872071162 0.00020796192966088256
-72.84854328680376 -9.022970417543307
63.825780831190116 0.00020796192966088256
-82.8421237122293 -10.249059999999993
72.5932716741575

for model in lshaped.submodel.values():
    print(model.relaxmod.status)
lshaped.submodel[1].relaxmod.computeIIS()
lshaped.submodel[1].relaxmod.write('./Reports/IIS/Infeasible.ilp')

In [8]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.variables.x[i,j,k].X > 0.5:
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges,bounds

AttributeError: Unable to retrieve attribute 'X'

In [ ]:
plot_trend(lshaped.upperbounds[1:],lshaped.lowerbounds[1:],0)

In [ ]:
lshaped.printsol(lshaped.submodel)

In [ ]:
(sum(lshaped.variables.h[i,j].X for i,j in lshaped.variables.h.keys()),
sum(lshaped.variables.p_l[j].X for j in lshaped.variables.p_l.keys()))

In [ ]:
lshaped.submodel[3].sim.alpha

In [ ]:
lshaped.model.ObjBound